In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [8]:
train_dl = torch.utils.data.DataLoader(dataset=datasets.MNIST('../data/mnist',
                                                              download=True,
                                                              train=True,
                                                              transform=transforms.Compose([
                                                                 transforms.ToTensor(),
                                                                 transforms.Normalize((0.1307,), (0.3081)),
                                                              ])),
                                                              batch_size=8,
                                                              shuffle=True,
                                                             )

0it [00:00, ?it/s]

9920512it [00:01, 6985108.74it/s]                             


Extracting ../data/mnist/MNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 102856.15it/s]                           
0it [00:00, ?it/s]

Extracting ../data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 1954338.52it/s]                            
0it [00:00, ?it/s]

Extracting ../data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 41108.08it/s]            

Extracting ../data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [9]:
test_dl = torch.utils.data.DataLoader(dataset=datasets.MNIST('../data/mnist',
                                                              download=True,
                                                              train=False,
                                                              transform=transforms.Compose([
                                                                 transforms.ToTensor(),
                                                                 transforms.Normalize((0.1307,), (0.3081)),
                                                              ])),
                                                              batch_size=8,
                                                              shuffle=True,
                                                             )

In [10]:
from torch.utils.data.dataset import Subset

In [11]:
# random splitting 
train_indices = ...
val_indices = ...

In [12]:
val_dl = Subset(train_dl, val_indices)

In [13]:
val_dl